In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
cudnn.benchmark = True
plt.ion()   # interactive mode

Roboflow for augmentation and resizing

In [1]:
!conda activate pytorch
import torch
from matplotlib import pyplot as plt
import numpy as np
torch.cuda.is_available()
import cv2
stream = cv2.VideoCapture(0) # 0 means read from local camera.
model = torch.hub.load('ultralytics/yolov5','yolov5s',pretrained=True)

Using cache found in C:\Users\boop/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-3-16 torch 1.10.2 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)



KeyboardInterrupt: 

In [4]:
!conda activate pytorch
import torch
from matplotlib import pyplot as plt
import numpy as np
torch.cuda.is_available()
import cv2
import winsound
stream = cv2.VideoCapture(0) # 0 means read from local camera.

In [23]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLOv5', np.squeeze(results.render()))
    
    if len(results.xywh[0].tolist()) != 0:
        conf_score = results.xywh[0].tolist()[0][-2]
        frequency = 2500  # Set Frequency To 2500 Hertz
        duration = 50  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
print(results.__dict__)
cap.release()
cv2.destroyAllWindows()

{'imgs': [array([[[187, 177, 153],
        [183, 173, 149],
        [183, 173, 149],
        ...,
        [167, 161, 149],
        [167, 161, 148],
        [166, 160, 147]],

       [[184, 174, 148],
        [181, 171, 145],
        [181, 171, 145],
        ...,
        [167, 161, 149],
        [166, 160, 147],
        [165, 159, 146]],

       [[186, 174, 147],
        [183, 171, 144],
        [183, 171, 144],
        ...,
        [167, 161, 148],
        [166, 160, 147],
        [165, 159, 146]],

       ...,

       [[128, 128, 128],
        [127, 127, 127],
        [127, 127, 127],
        ...,
        [177, 203, 251],
        [174, 200, 250],
        [172, 198, 248]],

       [[127, 127, 127],
        [126, 126, 126],
        [127, 127, 127],
        ...,
        [180, 204, 251],
        [178, 202, 251],
        [179, 203, 252]],

       [[127, 127, 127],
        [126, 126, 126],
        [126, 126, 126],
        ...,
        [181, 204, 250],
        [181, 205, 252],
        [183, 

In [8]:
!cd yolov5 && python train.py --img 416 --batch 16 --epochs 1000 --data data.yml --weights yolov5s.pt --workers 4

^C


In [13]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp9/weights/best.pt', force_reload=True)
model.conf = 0.6  # confidence threshold (0-1)
model.iou = 0.6  # NMS IoU threshold (0-1)  

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\boop/.cache\torch\hub\master.zip
YOLOv5  2022-3-26 torch 1.10.2 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [21]:
image = os.path.join('ultimate_test.png')
results = model(image)

In [19]:
image2 = os.path.join('couter_ex.jpg')
results2 = model(image2)

In [17]:
image3 = os.path.join('turtle5.jpg')
results3 = model(image3)

In [22]:
results.show()

In [46]:
results2.xywh[0].tolist()[0][-2]

0.6578324437141418

In [28]:
results.show()

In [8]:
results2.print()

image 1/1: 1494x1160 (no detections)
Speed: 41.0ms pre-process, 54.0ms inference, 1.0ms NMS per image at shape (1, 3, 640, 512)


In [20]:
dir(results)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'crop',
 'display',
 'files',
 'imgs',
 'n',
 'names',
 'pandas',
 'pred',
 'print',
 'render',
 's',
 'save',
 'show',
 't',
 'times',
 'tolist',
 'xywh',
 'xywhn',
 'xyxy',
 'xyxyn']

In [66]:
results2.xyxyn[0].tolist()

[]

TypeError: 'Detections' object is not subscriptable

In [4]:
import os
os.getcwd()

'C:\\Users\\boop\\pytorch_RTOD'

need to get confidence value and label and to control the beep with it

Customize and Transfer and Fine-Tune

In [13]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'hymenoptera_data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
class_names

['ants', 'bees']

In [20]:
dir(image_datasets)
image_datasets['train']

Dataset ImageFolder
    Number of datapoints: 244
    Root location: hymenoptera_data/hymenoptera_data\train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [16]:
device

device(type='cuda', index=0)

In [3]:
def score_frame(frame, model):
    """
    The function below identifies the device which is availabe 
    to make the prediction and uses it to load and infer the frame. 
    Once it has results it will extract the labels and cordinates(Along with scores) 
    for each object detected in the frame.
    """
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("The selected device is:", device)
    model.to(device)
    frame = [torch.tensor(frame)]
    results = model(frame)
    labels = results.xyxyn[0][:, -1].numpy()
    cord = results.xyxyn[0][:, :-1].numpy()
    return labels, cord

In [5]:
def plot_boxes(cv2results, frame):
    """
    The function below takes the results and the frame as input and plots
    boxes over all the objects which have a score higer than our threshold.
    """
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]
    for i in range(n):
        row = cord[i]
        # If score is less than 0.2 we avoid making a prediction.
        if row[4] < 0.2: 
            continue
        x1 = int(row[0]*x_shape)
        y1 = int(row[1]*y_shape)
        x2 = int(row[2]*x_shape)
        y2 = int(row[3]*y_shape)
        bgr = (0, 255, 0) # color of the box
        classes = self.model.names # Get the name of label index
        label_font = cv2.FONT_HERSHEY_SIMPLEX #Font for the label.
        cv2.rectangle(frame, \
                      (x1, y1), (x2, y2), \
                       bgr, 2) #Plot the boxes
        cv2.putText(frame,\
                    classes[labels[i]], \
                    (x1, y1), \
                    label_font, 0.9, bgr, 2) #Put a label over box.
        return frame

In [6]:
def __call__():
    """
    The Function below oracestrates the entire operation and performs the 
    real-time parsing for video stream.
    """
    player = stream #Get your video stream.
    #assert  # Make sure that their is a stream. 
    #Below code creates a new video writer object to write our
    #output stream.
    x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
    y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
    four_cc = cv2.VideoWriter_fourcc(*"MJPG") #Using MJPEG codex
    out = cv2.VideoWriter('out_file.mjpg', four_cc, 20, \
                          (x_shape, y_shape)) 
    ret, frame = player.read() # Read the first frame.
    while player.isOpened(): #rect: # Run until stream is out of frames
        start_time = time() # We would like to measure the FPS.
        results = score_frame(frame) # Score the Frame
        frame = plot_boxes(results, frame) # Plot the boxes.
        end_time = time()
        fps = 1/np.round(end_time - start_time, 3) #Measure the FPS.
        print(f"Frames Per Second : {fps}")
        out.write(frame) # Write the frame onto the output.
        ret, frame = player.read() # Read next frame.

In [ ]:
__call__()

In [ ]:
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False
    
import torch.nn as nn
# Add on classifier
model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))

model = model.to('cuda')

from torch import optim
# Loss and optimizer
criteration = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
model